In [12]:
import sqlite3
import pandas as pd 
import random
import os 
import numpy as np 
import ast 

# --- Set paths to database ---

os.chdir("/Users/duartedasilva/Desktop/Data Work/Data Projects/manga_anilist_pipeline")

BASE_DIR = os.getcwd()
DATA_PATH = os.path.join(BASE_DIR, "data", "top_manga2.parquet")
DB_PATH = os.path.join(BASE_DIR, "db", "top_manga2.db")

print("Resolved Path:", os.path.abspath(DATA_PATH))
print("File exists?:", os.path.exists(DATA_PATH))


Resolved Path: /Users/duartedasilva/Desktop/Data Work/Data Projects/manga_anilist_pipeline/data/top_manga2.parquet
File exists?: True


In [13]:
# --- Load data as CSV file ---

top_manga = pd.read_parquet(DATA_PATH)
print(type(top_manga["staff_names"].iloc[0]))
print(top_manga["staff_names"].head(3))
print("COLUMNS:",top_manga.columns)
print(top_manga.info)
print(top_manga.describe(include='all'))

print("Dataframe:", top_manga.head(20))


<class 'numpy.ndarray'>
0    [{'name': 'Kentarou Miura', 'primaryOccupation...
1    [{'name': 'Yasuki Tanaka', 'primaryOccupations...
2    [{'name': 'Takehiko Inoue', 'primaryOccupation...
Name: staff_names, dtype: object
COLUMNS: Index(['manga_id', 'title_romaji', 'title_english', 'title_native', 'format',
       'popularity', 'average_score', 'genres', 'start_date', 'end_date',
       'status', 'chapters', 'volumes', 'staff_names'],
      dtype='object')
<bound method DataFrame.info of       manga_id                                       title_romaji  \
0        30002                                            Berserk   
1        31706           JoJo no Kimyou na Bouken: Steel Ball Run   
2        30656                                           Vagabond   
3        30013                                          ONE PIECE   
4        64053  Umineko no Naku Koro ni Chiru Episode 8: Twili...   
...        ...                                                ...   
1195    134556          

In [14]:
# --- Handle NULLS --- 

print("BEFORE:",top_manga.isna().sum())
top_manga["title_english"] = top_manga["title_english"].fillna("Not Recorded").astype(str)
top_manga["chapters"] = top_manga["chapters"].fillna(0).astype(float)
top_manga["volumes"] = top_manga["volumes"].fillna(0).astype(float)

print("AFTER:",top_manga.isna().sum())

print("Staff:",top_manga["staff_names"].head(10))

BEFORE: manga_id           0
title_romaji       0
title_english    224
title_native       0
format             0
popularity         0
average_score      0
genres             0
start_date         0
end_date           0
status             0
chapters         388
volumes          474
staff_names        0
dtype: int64
AFTER: manga_id         0
title_romaji     0
title_english    0
title_native     0
format           0
popularity       0
average_score    0
genres           0
start_date       0
end_date         0
status           0
chapters         0
volumes          0
staff_names      0
dtype: int64
Staff: 0    [{'name': 'Kentarou Miura', 'primaryOccupation...
1    [{'name': 'Yasuki Tanaka', 'primaryOccupations...
2    [{'name': 'Takehiko Inoue', 'primaryOccupation...
3    [{'name': 'Eiichirou Oda', 'primaryOccupations...
4    [{'name': 'Kei Natsumi', 'primaryOccupations':...
5    [{'name': 'Naoki Urasawa', 'primaryOccupations...
6    [{'name': 'Makoto Yukimura', 'primaryOccupatio...
7    [{

In [15]:
# --- Transform numpy arrays into lists ---

top_manga["staff_names"] = top_manga["staff_names"].apply(
    lambda x: list(x) if isinstance(x, np.ndarray) else x
)


In [16]:
# --- Transform Authors Occupations to Lists --- 

def fix_occupations(staff_list):
    if not isinstance(staff_list, list):
        return staff_list
    fixed = []
    for p in staff_list:
        occ = p.get("primaryOccupations", [])
        if isinstance(occ, np.ndarray):
            p["primaryOccupations"] = list(occ)
        fixed.append(p)
    return fixed

top_manga["staff_names"] = top_manga["staff_names"].apply(fix_occupations)


In [17]:
# --- Normalize Staff & Create Dataframe --- 

staff_rows = [] 

for _, row in top_manga.iterrows():
    manga_id = row["manga_id"]
    staff_list = row["staff_names"]

    if not isinstance(staff_list, list):
        continue

    for person in staff_list:
        name = person.get("name")
        occupations = person.get("primaryOccupations")

        if not occupations:
            staff_rows.append({
                "manga_id": manga_id,
                "staff_name": name,
                "occupation": "Not Recorded"
            })
        else:
            for occ in occupations:
                staff_rows.append({
                    "manga_id": manga_id,
                    "staff_name": name,
                    "occupation": occ
                })

staff_df = pd.DataFrame(staff_rows) # Save to dataframe 
print(staff_df.head(5))

   manga_id      staff_name    occupation
0     30002  Kentarou Miura       Mangaka
1     30002      Kouji Mori       Mangaka
2     30002     Studio Gaga  Not Recorded
3     30002    Marc Bernabé    Translator
4     30002    Paweł Dybała    Translator


In [18]:
# --- Normalize Genres ---

print(type(top_manga["genres"].iloc[0]))
print(top_manga["genres"].head(5))

# Step 1: Flatten all genres with manga_id 

genres_flat = top_manga.explode("genres")[["manga_id", "genres"]].dropna()
genres_flat.columns = ["manga_id", "genres"]

print("Genres:", genres_flat)

<class 'numpy.ndarray'>
0    [Action, Adventure, Drama, Fantasy, Horror, Ps...
1    [Action, Adventure, Drama, Mystery, Supernatural]
2                           [Action, Adventure, Drama]
3                 [Action, Adventure, Comedy, Fantasy]
4              [Drama, Fantasy, Mystery, Supernatural]
Name: genres, dtype: object
Genres:       manga_id         genres
0        30002         Action
0        30002      Adventure
0        30002          Drama
0        30002        Fantasy
0        30002         Horror
...        ...            ...
1198    125624   Supernatural
1199    112978         Horror
1199    112978        Mystery
1199    112978  Psychological
1199    112978   Supernatural

[3876 rows x 2 columns]


In [19]:
# --- Handle Date Columns ---
from datetime import datetime

print(top_manga["start_date"], top_manga["end_date"])

def parse_date(date_dict):
    if not isinstance(date_dict, dict):
        return None
    
    year = date_dict.get('year')
    month = date_dict.get('month')
    day = date_dict.get('day')

    if not year:
        return None 
    
    # Default missing month/day to 1 
    try: 
        return datetime(int(year), int(month or 1), int(day or 1)).date().isoformat()
    except:
        return None 
    

# Apply function on columns 

top_manga["start_date"] = top_manga["start_date"].apply(parse_date)
top_manga["end_date"] = top_manga["end_date"].apply(parse_date)

# Verify changes 
print("AFTER:", top_manga["start_date"], top_manga["end_date"], top_manga["start_date"].dtypes)


0        {'day': 25.0, 'month': 8.0, 'year': 1989}
1        {'day': 19.0, 'month': 1.0, 'year': 2004}
2         {'day': 3.0, 'month': 9.0, 'year': 1998}
3        {'day': 22.0, 'month': 7.0, 'year': 1997}
4        {'day': 21.0, 'month': 1.0, 'year': 2012}
                           ...                    
1195     {'day': 22.0, 'month': 5.0, 'year': 2021}
1196      {'day': 9.0, 'month': 5.0, 'year': 2020}
1197      {'day': 1.0, 'month': 8.0, 'year': 2020}
1198    {'day': 10.0, 'month': 11.0, 'year': 2020}
1199    {'day': 30.0, 'month': 12.0, 'year': 2000}
Name: start_date, Length: 1200, dtype: object 0         {'day': None, 'month': None, 'year': None}
1        {'day': 19.0, 'month': 4.0, 'year': 2011.0}
2         {'day': None, 'month': None, 'year': None}
3         {'day': None, 'month': None, 'year': None}
4        {'day': 22.0, 'month': 6.0, 'year': 2015.0}
                            ...                     
1195      {'day': None, 'month': None, 'year': None}
1196    {'day': 26.0, 

In [20]:
print(len(top_manga))

1200


# DATABASE CREATION AND CONNECTION

In [21]:
print(top_manga.columns)
print(top_manga["chapters"])

Index(['manga_id', 'title_romaji', 'title_english', 'title_native', 'format',
       'popularity', 'average_score', 'genres', 'start_date', 'end_date',
       'status', 'chapters', 'volumes', 'staff_names'],
      dtype='object')
0        0.0
1       95.0
2        0.0
3        0.0
4       42.0
        ... 
1195     0.0
1196    86.0
1197    12.0
1198    90.0
1199     1.0
Name: chapters, Length: 1200, dtype: float64


In [23]:
# --- Create Database Connection ---

conn = sqlite3.connect(DB_PATH)
cur = conn.cursor()

# --- Create top_manga2 table ---

cur.executescript("""
                CREATE TABLE IF NOT EXISTS top_manga2(
                  manga_id INTEGER,
                  title_romaji TEXT,
                  title_english TEXT,
                  title_native TEXT, 
                  format TEXT,
                  popularity INTEGER,
                  average_score INTEGER,
                  start_date TEXT,
                  end_date TEXT, 
                  status TEXT,
                  chapters FLOAT,
                  volumes FLOAT,
                  PRIMARY KEY (manga_id)
                  );
""")

# Convert top_manga DataFrame into list of tuples

top_manga_tuples = top_manga[["manga_id", "title_romaji", "title_english", "title_native", "format", "popularity", "average_score", "start_date", "end_date", "status",
                              "chapters", "volumes"]].itertuples(index=False, name=None)

# Insert into database 
cur.execute("DELETE FROM top_manga2")

cur.executemany("""
                INSERT OR REPLACE INTO top_manga2 (manga_id, title_romaji, title_english, title_native, format, popularity, average_score, start_date, end_date, status,
                chapters, volumes) VALUES ( ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)""", top_manga_tuples)


cur.execute("SELECT * FROM top_manga2 LIMIT 20")
print(cur.fetchall())

[(30001, 'MONSTER', 'Monster', 'MONSTER', 'MANGA', 96925, 91, '1994-12-05', '2001-12-20', 'FINISHED', 162.0, 18.0), (30002, 'Berserk', 'Berserk', 'ベルセルク', 'MANGA', 221068, 93, '1989-08-25', None, 'RELEASING', 0.0, 0.0), (30003, '20 Seiki Shounen', '20th Century Boys', '20世紀少年', 'MANGA', 96329, 88, '1999-09-27', '2006-04-24', 'FINISHED', 249.0, 22.0), (30004, 'Yokohama Kaidashi Kikou', 'Yokohama Kaidashi Kikou', 'ヨコハマ買い出し紀行', 'MANGA', 33955, 86, '1994-04-26', '2006-02-26', 'FINISHED', 144.0, 14.0), (30007, 'Hajime no Ippo', 'Hajime no Ippo: Fighting Spirit!', 'はじめの一歩', 'MANGA', 32059, 87, '1989-09-27', None, 'RELEASING', 0.0, 0.0), (30009, 'Tsubasa: RESERVoir CHRoNiCLE', 'Tsubasa RESERVoir CHRoNiCLE', 'ツバサ -RESERVoir CHRoNiCLE-', 'MANGA', 16472, 80, '2003-05-21', '2009-10-07', 'FINISHED', 233.0, 28.0), (30010, 'xxxHolic', 'xxxHOLiC', '×××ホリック', 'MANGA', 21573, 82, '2003-02-24', '2011-02-09', 'FINISHED', 214.0, 19.0), (30011, 'NARUTO', 'Naruto', 'NARUTO -ナルト-', 'MANGA', 91214, 79, '1999-

In [24]:
# --- Create Staff Table --- 

cur.execute("DROP TABLE IF EXISTS staff_2")

cur.execute("""
            CREATE TABLE IF NOT EXISTS staff_2(
            manga_id INTEGER,
            staff_name TEXT,
            occupation TEXT, 
            PRIMARY KEY (manga_id, staff_name, occupation),
            FOREIGN KEY (manga_id) references top_manga2(manga_id)
            );
""")

staff_tuples = staff_df[["manga_id", "staff_name", "occupation"]].itertuples(index=False, name=None)

# Fallback deletion 

cur.execute("DELETE FROM staff_2")

# Load into database 

cur.executemany("""
                INSERT OR REPLACE INTO staff_2(manga_id, staff_name, occupation) VALUES (?, ?, ?)""", staff_tuples)

cur.execute("SELECT * FROM staff_2 LIMIT 15")
print(cur.fetchall())

[(30002, 'Kentarou Miura', 'Mangaka'), (30002, 'Kouji Mori', 'Mangaka'), (30002, 'Studio Gaga', 'Not Recorded'), (30002, 'Marc Bernabé', 'Translator'), (30002, 'Paweł Dybała', 'Translator'), (30002, 'Drik Sada', 'Translator'), (30002, 'Duane Johnson', 'Translator'), (30002, 'Jason DeAngelis', 'Translator'), (30002, 'Jason DeAngelis', 'Writer'), (30002, 'David Deleule', 'Translator'), (30002, 'René-Gilles Deberdt', 'Translator'), (30002, 'Dan Nakrosis', 'Letterer'), (30002, 'John Schmitt-Weigand', 'Translator'), (30002, 'Diego Rodeguero', 'Not Recorded'), (30002, 'Alp İlkkurşun', 'Translator')]


In [25]:
# --- Create genres table --- 

# Fallback deletion 

cur.execute("DROP TABLE IF EXISTS genres_2")

cur.execute("""CREATE TABLE IF NOT EXISTS genres_2(
            manga_id INTEGER,
            genre TEXT,
            PRIMARY KEY (manga_id, genre),
            FOREIGN KEY (manga_id) REFERENCES top_manga2(manga_id)
            );
""")

genre_tuples = genres_flat[["manga_id", "genres"]].itertuples(index=False, name=None)

cur.executemany("INSERT OR REPLACE INTO genres_2 (manga_id, genre) VALUES (?, ?)", genre_tuples)
cur.execute("SELECT * FROM genres_2 LIMIT 10")
print(cur.fetchall())

[(30002, 'Action'), (30002, 'Adventure'), (30002, 'Drama'), (30002, 'Fantasy'), (30002, 'Horror'), (30002, 'Psychological'), (31706, 'Action'), (31706, 'Adventure'), (31706, 'Drama'), (31706, 'Mystery')]


In [26]:
print(genres_flat["genres"].unique())
print(len(top_manga))

['Action' 'Adventure' 'Drama' 'Fantasy' 'Horror' 'Psychological' 'Mystery'
 'Supernatural' 'Comedy' 'Thriller' 'Sports' 'Sci-Fi' 'Slice of Life'
 'Romance' 'Music' 'Mecha' 'Ecchi' 'Mahou Shoujo' 'Hentai']
1200


In [27]:
# --- Commit & Close ---

conn.commit()
conn.close()

In [28]:
DB_PATH_2 = os.path.join(BASE_DIR, "db", "top_manga.db")

conn = sqlite3.connect(DB_PATH_2)
cur = conn.cursor()

top_manga_original = pd.read_sql("SELECT * FROM top_manga", conn)
print(len(top_manga_original))


301
